In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
##import glob
##import errno
import re
import math
import numpy as np

##path = 'documents/download/*.txt'
##files = glob.glob(path)
##
##for name in files:
##    try:
##        with open(name) as f:
##            print(name)
##    except IOError as exc:
##        if exc.errno != errno.EISDIR:
##            raise

REGEX = re.compile(r"\s")
def tokenize(text):
    return [tok.strip().lower() for tok in REGEX.split(text)]

ps = PorterStemmer()
stop_words = set(stopwords.words("english"))

# open all passages
text = []
file = open("documents/text1.txt","r");
text1 = file.read()
text.append(text1)
file = open("documents/text2.txt","r");
text2 = file.read()
text.append(text2)
file = open("documents/text3.txt","r");
text3 = file.read()
text.append(text3)
file = open("documents/text4.txt","r");
text4 = file.read()
text.append(text4)
##print(text)
##print(len(text))


# dibuat huruf kecil semua
text = [s.lower() for s in text]
##print(text)


# remove all punctuations
punctuation = ["." , "," , "(" , ")" , "%"]
i = 0
for row in text:
    text[i] = ''.join([p for p in text[i] if p not in punctuation])
    i += 1
##print("-----text-----")
##print(text)


# tokenize the article into words
i = 0
words = [];
for row in text:
    words.append(word_tokenize(text[i]))
    i += 1
##print("-----words-----")
##print(words)

                             
# filter the stopwords
i = 0
filteredText = [];
for row in words:
    filteredText.append([w for w in words[i] if not w in stop_words])
    i += 1
##print("-----filtered-----")
##print(filteredText)


# stem every word
i = 0
stemmedWords = [];
for row in filteredText:
    stemmedWords.append([ps.stem(w) for w in filteredText[i]])
    i += 1
print("-----stemmedWords-----")
print(stemmedWords)


train_set = [' '.join(w) for w in stemmedWords]
print("-----train set-----")
print(train_set)


count_vectorizer = CountVectorizer(tokenizer=tokenize)
data = count_vectorizer.fit_transform(train_set).toarray()
vocab = count_vectorizer.get_feature_names()

print ("-----Term FREQUENCY-----")
print (data)
print ("-----VECTOR FITUR-----")
print (vocab)
print ("-----NUMBER VECTOR FEATURES-----")
print (len(vocab))


# export excel terms Frequency
xls="Term-Frequency,"
for vc in vocab:
	xls+=vc+","

xls += "\n"

# print terms Frequency for each document
i = 0
for row in data:
	xls += "Document "+str(i)+","
	for col in row:
		xls += str(col)+","
	xls += "\n"	
	i += 1



# export excel Document Frequency
xls += "\n\n\n"
xls += "Document-Frequency,"
for vc in vocab:
	xls+=vc+","

xls += "\n"

dfs = []
for row in data:
	i = 0
	xls += ","
	for col in row:
		count = 0
		if data[0][i]>0:count += 1
		if data[1][i]>0:count += 1
		if data[2][i]>0:count += 1
		if data[3][i]>0:count += 1
		dfs.append(count)
		xls += str(count)+","
		i += 1
	xls += "\n"
	break

print("-----dfs-----")
print(dfs)

### sort document Frequency
##xls += "\n\n\n"
##xls += "Document-Frequency,"

# export TF-IDF
xls += "\n\n\n"
xls += "TF-IDF,"
for vc in vocab:
	xls+=vc+","

xls += "\n"

i=0
TFIDF=[]
for row in data:
	xls += "DOCUMENT "+str(i)+","
	j=0
	data_tfidf=[]
	for col in row:
		# calculate tfidf with formula tfidf = tf*log(Ndocument/df)
		tfidf = col*math.log(4/dfs[j])
		data_tfidf.append(tfidf)
		xls += str(tfidf)+","
		j += 1
	
	TFIDF.append(data_tfidf)
	xls += "\n"
	i += 1

xls += "\n\n\n"
xls+= ","
for i in range(4):
	xls += "Document"+str(i+1)+","
xls += "\n"

for i in range(4):
	xls += "Document"+str(i+1)+","
	for j in range(4):
		dt1 = TFIDF[i]
		dt2 = TFIDF[j]
		# calculate dot product
		dotproduct = np.dot(dt1,dt2)
		# calculate 2 data magnitude
		magnitude1 = np.sqrt(np.dot(dt1,dt1))
		magnitude2 = np.sqrt(np.dot(dt2,dt2))

		cos_similarity = dotproduct/(magnitude1*magnitude2)
		xls += str(cos_similarity)+","
	xls += "\n"


##file = open("tf-idf.csv","w")
##file.write(xls)
##print ("file created")

##new query
print("-----query----")
file = open("documents/follower.txt","r");
query = file.read()
query = query.lower()
query = ''.join([p for p in query if p not in punctuation])
tokenizedQuery = word_tokenize(query)
filteredQuery = [w for w in tokenizedQuery if not w in stop_words]
stemmedQuery = [ps.stem(w) for w in filteredQuery]
joinedQuery = [' '.join(stemmedQuery)]

dataQuery = count_vectorizer.fit_transform(joinedQuery).toarray()
vocabQuery = count_vectorizer.get_feature_names()

# print terms Frequency for query
xls += "\n\n\n"
xls += "Query Term-Frequency,"
for vc in vocabQuery:
	xls+=vc+","

xls += "\n"
for row in dataQuery:
	xls += "Query " + ","
	
	for col in row:
		xls += str(col)+","
		
# export TF-IDF
xls += "\n\n\n"
xls += "Query TF-IDF,"
for vc in vocabQuery:
	xls+=vc+","

xls += "\n"

QueryTFIDF=[0]*len(TFIDF[0])
xls += "Query "+","
j=0
query_data_tfidf=[]
queryTfidf = 0;
for col in dataQuery[0]:
    # calculate tfidf with formula tfidf = tf*log(Ndocument/df)
    queryTfidf = col*math.log(4/dfs[vocab.index(vocabQuery[j])])
    query_data_tfidf.append(queryTfidf)
    xls += str(queryTfidf) + ","

    j += 1

#find index in vocab
i=0
for col in vocabQuery:
    QueryTFIDF[vocab.index(col)] = query_data_tfidf[i]
    i += 1
    
xls += "\n\n\n"
i=0
xls+= ","
for i in range(4):
	xls += "Document"+str(i+1)+","
xls += "\n"

xls += "Query"+","
cos_similarity=[]
i=0
for j in range(4):
    dt1 = QueryTFIDF
    dt2 = TFIDF[j]
    # calculate dot product
    dotproduct = np.dot(dt1,dt2)
    # calculate 2 data magnitude
    magnitude1 = np.sqrt(np.dot(dt1,dt1))
    magnitude2 = np.sqrt(np.dot(dt2,dt2))

    cos_similarity.append(dotproduct/(magnitude1*magnitude2))
    xls += str(cos_similarity[i])+","
    i += 1

	
file = open("tf-idf.csv","w")
file.write(xls)
print ("file created")
print ("Query is similar to document:", cos_similarity.index(max(cos_similarity))+1)


-----stemmedWords-----
[['charact', 'segment', 'handwritten', 'manuscript', 'often', 'present', 'complic', 'task', 'mani', 'factor', 'caus', 'segment', 'difficult', 'inconsist', 'slope', 'slant', 'length', 'width', 'charact', 'well', 'intersect', 'two', 'charact', 'either', 'differ', 'line', 'paper', 'propos', 'new', 'approach', 'combin', 'statist', 'structur', 'analys', 'gener', 'javanes', 'script', 'line', 'segment', 'javanes', 'manuscript', 'imag', 'everi', 'time', 'new', 'manuscript', 'discov', 'object', 'make', 'charact', 'manuscript', 'identifi', 'use', 'interconnect', 'oper', 'identifi', 'compon', 'script', 'object', 'interconnect', 'given', 'label', 'next', 'task', 'calcul', 'averag', 'height', 'averag', 'width', 'object', 'given', 'label', 'standard', 'deviat', 'inform', 'use', 'guid', 'averag', 'normal', 'script', 'ie', 'charact', 'height', 'width', 'exce', 'averag', 'valu', 'plu', 'standard', 'deviat', 'conclud', 'charact', 'question', 'fact', 'consist', 'two', 'charact', 't

ValueError: '!' is not in list